<h3>Multiword expressions identification and extraction</h3>
<p>Konrad Przewłoka</p>

<h4>Imports</h4>

In [96]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import os
import random
import math
import collections
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import requests

<h4>Load data</h4>

In [97]:
data=[]
files = os.listdir("../ustawy")
for file in files:
    with open("../ustawy" + '/' + file, 'r', encoding='utf8') as f:
        tmp = f.read().lower()
        data.append(tmp)

<h4>SpaCy tokenization</h4>

In [98]:
nlp = Polish()
tokenizer = nlp.tokenizer
tokenized_data = [[t.text for t in tokenizer(r)] for r in data ]
tokens_counter = collections.Counter([element for sublist in tokenized_data for element in sublist])
tmp = []
for key in tokens_counter.keys():
    if not key.isalpha():
        tmp.append(key)
        
for key in tmp:
    del tokens_counter[key]
    
tokens_total = sum(tokens_counter.values())

<h4>Bigram computation</h4>

In [99]:
bigrams=[]
for file in tokenized_data:
    previous_token = None
    for token in file:
        if previous_token != None:
            bigrams.append(" ".join([previous_token,token]))
        previous_token = token
bigrams_counter = collections.Counter(bigrams)

#Filter out bigrams containing characters other than letters
def is_correct(s):
    if len(s.split())!=2:
        return False
    for element in s:
        if not element.isalpha() and element!=' ':
            return False
    return True

tmp = []
for key in bigrams_counter.keys():
    if not is_correct(key):
        tmp.append(key)
        
for key in tmp:
    del bigrams_counter[key]

<h4>PMI for all bigrams</h4>

In [100]:
bigrams_total= sum(bigrams_counter.values())
def pmi(bigram):
    p_w1 = tokens_counter[bigram.split()[0]]/float(tokens_total)
    p_w2 = tokens_counter[bigram.split()[1]]/float(tokens_total)
    p_bigram = bigrams_counter[bigram]/bigrams_total
    return math.log2(p_bigram/(p_w1*p_w2))
bigrams_pmi = {bigram: pmi(bigram) for bigram  in bigrams_counter.keys()}
collections.Counter(bigrams_pmi).most_common()[:10]

[('kołowe jednoosiowe', 22.2954635058116),
 ('zbrojeń żelbeto', 22.2954635058116),
 ('prefabrykatów wnętrzowe', 22.2954635058116),
 ('gołe aluminiowe', 22.2954635058116),
 ('polistyrenu spienionego', 22.2954635058116),
 ('objaśnieniem figur', 22.2954635058116),
 ('wkładzie wnoszonym', 22.2954635058116),
 ('doktorem habilitowanym', 22.2954635058116),
 ('losy loteryjne', 22.2954635058116),
 ('ugaszone zapałki', 22.2954635058116)]

<h4>PMI for bigrams with 5 or more occurences</h4>

In [102]:
bigrams_filltered_pmi = {bigram: pmi(bigram) for bigram,count  in bigrams_counter.items() if count >=5}
collections.Counter(bigrams_filltered_pmi).most_common()[:10]

[('świeckie przygotowujące', 19.97353541092424),
 ('klęskami żywiołowymi', 19.97353541092424),
 ('ręcznego miotacza', 19.97353541092424),
 ('stajnią wyścigową', 19.97353541092424),
 ('otworami wiertniczymi', 19.97353541092424),
 ('obcowania płciowego', 19.97353541092424),
 ('młyny kulowe', 19.97353541092424),
 ('młynki młotkowe', 19.97353541092424),
 ('zaszkodzić wynikom', 19.97353541092424),
 ('grzegorz schetyna', 19.97353541092424)]

<h4>KNNRT tagging and lemmatization<h4>

In [15]:
taged_data = [(lambda text: requests.post("http://localhost:9200", data=text.encode("utf-8")).content.decode("utf-8"))(text) for text in data]

<h4>Bigram computation for tagged data<h4>

In [103]:
tokenized_data_tagged=[]
for file in taged_data:
    lines = file.split("\n")
    splits = [line.strip().split('\t') for line in lines if line.startswith("\t")]
    tokens = [(split[0].lower(),split[1].split(":")[0]) for split in splits if len(split)>=2]
    tokenized_data_tagged.append(tokens)

bigrams_tagged = [] 
for file in tokenized_data_tagged:
    prev_token = None
    file_bigrams=[]
    for token in file:
        if prev_token!=None:
            file_bigrams.append((prev_token,token))
        prev_token=token
    bigrams_tagged.append(file_bigrams)
tokens_total_tagged=[]    
for file in tokenized_data_tagged:
    for token in file:
        if token[0].isalpha():
            tokens_total_tagged.append(token)

tokens_counter_tagged= collections.Counter(tokens_total_tagged)
bigram_counter_tagged= collections.Counter([element for sublist in bigrams_tagged for element in sublist])

tmp = []
for key in bigram_counter_tagged.keys():
    if not key[0][0].isalpha() or not key[1][0].isalpha():
        tmp.append(key)
        
for key in tmp:
    del bigram_counter_tagged[key]

tokens_total_tagged = sum(tokens_counter_tagged.values())

<h4>PMI for all bigrams from tagged data<h4>

In [104]:
bigrams_total_tagged= sum(bigram_counter_tagged.values())
def pmi(bigram):
    p_w1 = tokens_counter_tagged[bigram[0]]/float(tokens_total_tagged)
    p_w2 = tokens_counter_tagged[bigram[1]]/float(tokens_total_tagged)
    p_bigram = bigram_counter_tagged[bigram]/bigrams_total_tagged
    return math.log2(p_bigram/(p_w1*p_w2))
bigrams_pmi_tagged = {bigram: pmi(bigram) for bigram  in bigram_counter_tagged.keys()}
collections.Counter(bigrams_pmi_tagged).most_common()[:10]

[((('tornister', 'subst'), ('nieskórzany', 'adj')), 22.151063128973618),
 ((('cji', 'interj'), ('wadociągowych', 'adj')), 22.151063128973618),
 ((('zbrojenia', 'subst'), ('żelbeto', 'subst')), 22.151063128973618),
 ((('reduktor', 'subst'), ('membranowy', 'adj')), 22.151063128973618),
 ((('rozdzielnice', 'adj'), ('prefabrykat', 'subst')), 22.151063128973618),
 ((('prefabrykat', 'subst'), ('wnętrzowy', 'adj')), 22.151063128973618),
 ((('polistyren', 'subst'), ('spienić', 'ppas')), 22.151063128973618),
 ((('uw', 'subst'), ('zględnieniu', 'subst')), 22.151063128973618),
 ((('któ', 'adj'), ('rych', 'adj')), 22.151063128973618),
 ((('zaniedbać', 'ppas'), ('wychowawczo', 'adv')), 22.151063128973618)]

<h4>PMI for bigrams with 5 or more occurences from tagged data</h4>

In [105]:
bigrams_filltered_pmi_tagged = {bigram: pmi(bigram) for bigram,count  in bigram_counter_tagged.items() if count >=5}
collections.Counter(bigrams_filltered_pmi_tagged).most_common()[:10]

[((('młynek', 'subst'), ('młotkowy', 'adj')), 19.829135034086256),
 ((('grzegorz', 'subst'), ('schetyna', 'subst')), 19.829135034086256),
 ((('pasta', 'subst'), ('emulsyjny', 'adj')), 19.566100628252464),
 ((('chrom', 'subst'), ('sześciowartościowy', 'adj')), 19.566100628252464),
 ((('adam', 'subst'), ('mickiewicz', 'subst')), 19.566100628252464),
 ((('piotrków', 'subst'), ('trybunalski', 'adj')), 19.343708206916016),
 ((('młyn', 'subst'), ('kulowy', 'adj')), 19.343708206916016),
 ((('środa', 'subst'), ('wlkp', 'brev')), 19.343708206916016),
 ((('przeponowy', 'adj'), ('rurowy', 'adj')), 19.151063128973618),
 ((('skrzynka', 'subst'), ('podawczy', 'adj')), 19.151063128973618)]

<h4>Untagged trigram computation</h4>

In [106]:
trigrams=[]
for file in tokenized_data:
    previous_token = None
    previous_previous_token = None
    for token in file:
        if previous_token != None and previous_previous_token!=None:
            trigrams.append(" ".join([previous_previous_token,previous_token,token]))
        previous_previous_token = previous_token
        previous_token = token
trigrams = collections.Counter(trigrams)

def is_correct(s):
    if len(s.split())!=3:
        return False
    for element in s:
        if not element.isalpha() and element!=' ':
            return False
    return True

tmp = []
for key in trigrams.keys():
    if not is_correct(key):
        tmp.append(key)
        
for key in tmp:
    del trigrams[key]
    
filltered_trigrams = {trigram: count for trigram, count  in trigrams.items() if count>=5} 

<h4>Tagged trigram computation<h4>

In [107]:
trigrams_tagged=[]
for file in tokenized_data_tagged:
    previous_token = None
    previous_previous_token = None
    for token in file:
        if previous_token != None and previous_previous_token!=None:
            trigrams_tagged.append((previous_previous_token,previous_token,token))
        previous_previous_token = previous_token
        previous_token = token
trigrams_tagged = collections.Counter(trigrams_tagged)

tmp = []
for key in trigrams_tagged.keys():
    if not key[0][0].isalpha() or not key[1][0].isalpha() or not key[2][0].isalpha():
        tmp.append(key)
        
for key in tmp:
    del trigrams_tagged[key]
    
filltered_trigrams_tagged = {trigram: count for trigram, count  in trigrams_tagged.items() if count>=5} 

<h4>PMI for trigrams with 5 or more occurences from untagged data</h4>

In [108]:
total_trigrams=sum(trigrams.values())
def pmi3(trigram):
    p_w1w2 = bigrams_counter[" ".join([trigram.split()[0],trigram.split()[1]])]/bigrams_total
    p_w2w3 = bigrams_counter[" ".join([trigram.split()[1],trigram.split()[2]])]/bigrams_total
    p_trigram = filltered_trigrams[trigram]/total_trigrams
    return math.log(p_trigram/(p_w1w2*p_w2w3))
filltered_trigrams_pmi = {trigram: pmi3(trigram) for trigram  in filltered_trigrams.keys()}
collections.Counter(filltered_trigrams_pmi).most_common()[:10]

[('uroczyście na powierzonym', 13.432253789295611),
 ('na powierzonym mi', 13.432253789295611),
 ('zostało im cofnięte', 13.432253789295611),
 ('im cofnięte pozwolenie', 13.432253789295611),
 ('w magazynie celnym', 13.432253789295611),
 ('zanim zapadł pierwszy', 13.432253789295611),
 ('zapadł pierwszy wyrok', 13.432253789295611),
 ('do ciągu przestępstw', 13.432253789295611),
 ('dwukrotnie użyty wyraz', 13.432253789295611),
 ('funduszu odrębny bilans', 13.432253789295611)]

<h4>PMI for trigrams with 5 or more occurences from tagged data</h4>

In [109]:
total_trigrams_tagged=sum(trigrams.values())
def pmi3(trigram):
    p_w1w2 = bigram_counter_tagged[(trigram[0],trigram[1])]/bigrams_total_tagged
    p_w2w3 = bigram_counter_tagged[(trigram[1],trigram[2])]/bigrams_total_tagged
    p_trigram = filltered_trigrams_tagged[trigram]/total_trigrams_tagged
    return math.log(p_trigram/(p_w1w2*p_w2w3))
filltered_trigrams_pmi_tagged = {trigram: pmi3(trigram) for trigram  in filltered_trigrams_tagged.keys()}
collections.Counter(filltered_trigrams_pmi_tagged).most_common()[:10]

[((('cel', 'subst'), ('zastrzec', 'ger'), ('pierwszeństwo', 'subst')),
  13.650222204966445),
 ((('uroczyście', 'adv'), ('na', 'prep'), ('powierzyć', 'ppas')),
  13.650222204966445),
 ((('przedstawiać', 'fin'), ('korzystny', 'adj'), ('bilans', 'subst')),
  13.650222204966445),
 ((('wnieść', 'ger'), ('nowy', 'adj'), ('wadium', 'subst')),
  13.650222204966445),
 ((('zanim', 'comp'), ('zapaść', 'praet'), ('pierwszy', 'adj')),
  13.650222204966445),
 ((('wpływ', 'subst'), ('uchybienie', 'subst'), ('na', 'prep')),
  13.650222204966445),
 ((('fundusz', 'subst'), ('odrębny', 'adj'), ('bilans', 'subst')),
  13.650222204966445),
 ((('każdy', 'adj'), ('rozliczać', 'ppas'), ('miesiąc', 'subst')),
  13.650222204966445),
 ((('usługa', 'subst'), ('korzystać', 'fin'), ('jednostka', 'subst')),
  13.650222204966445),
 ((('obowiązkowy', 'adj'), ('obciążyć', 'ger'), ('wynik', 'subst')),
  13.650222204966445)]

<h4>PMI for trigrams with 5 or more occurences from tagged data with additional filters (in this case adjective noun adjective)</h4>

In [110]:
collections.Counter({trigram:pmi for trigram,pmi in filltered_trigrams_pmi_tagged.items() if trigram[0][1]=='adj' and trigram[1][1]=='subst' and trigram[2][1]=='adj'}).most_common()[:10]

[((('czechosłowacki', 'adj'),
   ('republika', 'subst'),
   ('socjalistyczny', 'adj')),
  13.650222204966445),
 ((('porcelanowy', 'adj'), ('młyn', 'subst'), ('kulowy', 'adj')),
  13.650222204966445),
 ((('elektroniczny', 'adj'), ('skrzynka', 'subst'), ('podawczy', 'adj')),
  13.650222204966445),
 ((('ratowniczy', 'adj'), ('centrum', 'subst'), ('koordynacyjny', 'adj')),
  13.46790064817249),
 ((('wojskowy', 'adj'), ('areszt', 'subst'), ('dyscyplinarny', 'adj')),
  13.46790064817249),
 ((('pełny', 'adj'), ('krew', 'subst'), ('angielski', 'adj')),
  13.46790064817249),
 ((('bezwodny', 'adj'), ('tłuszcz', 'subst'), ('mleczny', 'adj')),
  13.46790064817249),
 ((('wojewódzki', 'adj'), ('zgromadzenie', 'subst'), ('wyborczy', 'adj')),
  13.46790064817249),
 ((('roczny', 'adj'), ('przygotowanie', 'subst'), ('przedszkolny', 'adj')),
  13.46790064817249),
 ((('krajowy', 'adj'), ('informacja', 'subst'), ('skarbowy', 'adj')),
  13.46790064817249)]

<h4>Table for bigrams<h4>

In [112]:
r1 = collections.Counter(bigrams_filltered_pmi).most_common()
r2 = [ x[0][0][0]+":"+x[0][0][1]+" "+x[0][1][0]+":"+x[0][1][1]+" "+str(x[1]) for x in collections.Counter(bigrams_pmi_tagged).most_common()]
t = PrettyTable(['PMI untagged', 'PMI tagged'])
for i in range(10):
    t.add_row([r1[i],r2[i]])
print(t)

+------------------------------------------------+-------------------------------------------------------+
|                  PMI untagged                  |                       PMI tagged                      |
+------------------------------------------------+-------------------------------------------------------+
| ('świeckie przygotowujące', 19.97353541092424) |   tornister:subst nieskórzany:adj 22.151063128973618  |
|  ('klęskami żywiołowymi', 19.97353541092424)   |    cji:interj wadociągowych:adj 22.151063128973618    |
|    ('ręcznego miotacza', 19.97353541092424)    |    zbrojenia:subst żelbeto:subst 22.151063128973618   |
|    ('stajnią wyścigową', 19.97353541092424)    |    reduktor:subst membranowy:adj 22.151063128973618   |
|  ('otworami wiertniczymi', 19.97353541092424)  | rozdzielnice:adj prefabrykat:subst 22.151063128973618 |
|   ('obcowania płciowego', 19.97353541092424)   |   prefabrykat:subst wnętrzowy:adj 22.151063128973618  |
|      ('młyny kulowe', 19.9735354109

<h4>Table for trigrams</h4>

In [90]:
r1 = collections.Counter(filltered_trigrams_pmi).most_common()
r2 = [ x[0][0][0]+":"+x[0][0][1]+" "+x[0][1][0]+":"+x[0][1][1]+" "+x[0][2][0]+":"+x[0][2][1]+" "+str(x[1]) for x in collections.Counter(filltered_trigrams_pmi_tagged).most_common()]
t = PrettyTable(['PMI untagged', 'PMI tagged'])
for i in range(10):
    t.add_row([r1[i],r2[i]])
print(t)

+---------------------------------------------------+----------------------------------------------------------------+
|                    PMI untagged                   |                           PMI tagged                           |
+---------------------------------------------------+----------------------------------------------------------------+
| ('uroczyście na powierzonym', 13.432253789295611) | cel:subst zastrzec:ger pierwszeństwo:subst 13.650222204966445  |
|     ('na powierzonym mi', 13.432253789295611)     |    uroczyście:adv na:prep powierzyć:ppas 13.650222204966445    |
|    ('zostało im cofnięte', 13.432253789295611)    | przedstawiać:fin korzystny:adj bilans:subst 13.650222204966445 |
|   ('im cofnięte pozwolenie', 13.432253789295611)  |      wnieść:ger nowy:adj wadium:subst 13.650222204966445       |
|     ('w magazynie celnym', 13.432253789295611)    |    zanim:comp zapaść:praet pierwszy:adj 13.650222204966445     |
|   ('zanim zapadł pierwszy', 13.432253789295611

<h4>Additinal table for tagged bigrams with and without aditional filter (adj-subst-adj)</h4>

In [95]:
r1 = [ x[0][0][0]+":"+x[0][0][1]+" "+x[0][1][0]+":"+x[0][1][1]+" "+x[0][2][0]+":"+x[0][2][1]+" "+str(round(x[1],4)) for x in collections.Counter({trigram:pmi for trigram,pmi in filltered_trigrams_pmi_tagged.items() if trigram[0][1]=='adj' and trigram[1][1]=='subst' and trigram[2][1]=='adj'}).most_common()]
r2 = [ x[0][0][0]+":"+x[0][0][1]+" "+x[0][1][0]+":"+x[0][1][1]+" "+x[0][2][0]+":"+x[0][2][1]+" "+str(round(x[1],4)) for x in collections.Counter(filltered_trigrams_pmi_tagged).most_common()]
t = PrettyTable(['PMI tagged with filter', 'PMI tagged'])
for i in range(10):
    t.add_row([r1[i],r2[i]])
print(t)

+---------------------------------------------------------------+-----------------------------------------------------+
|                     PMI tagged with filter                    |                      PMI tagged                     |
+---------------------------------------------------------------+-----------------------------------------------------+
| czechosłowacki:adj republika:subst socjalistyczny:adj 13.6502 |  cel:subst zastrzec:ger pierwszeństwo:subst 13.6502 |
|         porcelanowy:adj młyn:subst kulowy:adj 13.6502         |    uroczyście:adv na:prep powierzyć:ppas 13.6502    |
|     elektroniczny:adj skrzynka:subst podawczy:adj 13.6502     | przedstawiać:fin korzystny:adj bilans:subst 13.6502 |
|     ratowniczy:adj centrum:subst koordynacyjny:adj 13.4679    |       wnieść:ger nowy:adj wadium:subst 13.6502      |
|      wojskowy:adj areszt:subst dyscyplinarny:adj 13.4679      |     zanim:comp zapaść:praet pierwszy:adj 13.6502    |
|           pełny:adj krew:subst angiels

<h4>Answers</h4>
<h5>I</h5>
<p>We have to filter bigrams rather than token sequences in order to preserve real and meaningful connections between tokens. For example if we have a sequence of tokens looking like this: a1 a2 n1 n2 a3 a4, where a&lt;nr&gt; represents a valid token and n&lt;nr&gt; represents an invalid token, if we filter tokens first we will get bigrams: a1a2 a2a3 a3a4 whereas if we first create bigrams and filter then we will get bigrams a1a2 a3a4. As we see the operation order is important as we can end up with different bigrams depending of the order of operations. Filtering after creating bigrams is preferred as it preserves original words collocations.</p>
<h5>II</h5>
<p>It is hard to determine if lemmatization and tagging makes a significant difference in terms of quality of found results both for bigrams and trigrams. However lemmatization and tagging may offer us further insight of the data and enable additional filtering when searchin for specific expressions.</p>
<h5>III</h5>
<p>Those methods help in discovering multi word expressions of different types such as multiword named entities and multiword terms.</p>
<h5>IV</h5>
<p>It is definitely possible to devise better solutions especially for certain types of datasets. Depending on the task given and the understanding of the data it may be possible to devise better or additional filters other than non-letter characters. As an example in the last table of the exercise, simple tagged trigrams have been compared with the same trigrams but with additional filters that checked if the trigram was constructed from an adjective noun and adjective in that order. In my opinion expressions found with the additional filter are of higher potential quality than the unfiltered ones.</p>